In [14]:
import pandas as pd
import pickle
import glob

# Replace with your directory containing pickle files
pickle_files = glob.glob('/eos/uscms/store/user/aaportel/boostedhiggs/dec_5_top_2016_2016/TTToSemiLeptonic/outfiles/*.pkl')

# Load and store all dictionaries
dictionaries = []
for file in pickle_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
        dictionaries.append(data)


In [17]:
# Initialize a dictionary to hold the sums
sums = {
    'mu': { 'Trigger': 0, 'bMatch': 0, 'twoFatJet': 0, 'oneLep': 0, 'LepMatch': 0, 'HadMatch': 0, 'OneLep': 0 },
    'ele': { 'Trigger': 0, 'bMatch': 0, 'twoFatJet': 0, 'oneLep': 0, 'LepMatch': 0, 'HadMatch': 0, 'OneLep': 0 }
}

# Iterate over each item in the list
for item in dictionaries:
    # Assuming there is only one key per item and it is 'TTToSemiLeptonic'
    year_data = item['TTToSemiLeptonic']['2016']['cutflows']
    
    # Sum up the values for 'mu' and 'ele'
    for lepton in ['mu', 'ele']:
        for key in year_data[lepton]:
            sums[lepton][key] += year_data[lepton][key]

# Print the result
sums

{'mu': {'Trigger': 2710683.0,
  'bMatch': 573591.0,
  'twoFatJet': 90160.0,
  'oneLep': 72510.0,
  'LepMatch': 49585.0,
  'HadMatch': 5717.0,
  'OneLep': 3891.0},
 'ele': {'Trigger': 1982680.0,
  'bMatch': 415892.0,
  'twoFatJet': 72616.0,
  'oneLep': 50005.0,
  'LepMatch': 35176.0,
  'HadMatch': 4206.0,
  'OneLep': 3631.0}}

In [22]:
# Assuming 'sums' is the dictionary obtained from the previous step
# Convert 'sums' into DataFrames
df_mu = pd.DataFrame.from_dict(sums['mu'], orient='index', columns=['Count'])
df_ele = pd.DataFrame.from_dict(sums['ele'], orient='index', columns=['Count'])

# Function to calculate cumulative and relative efficiency
def calculate_efficiencies(df):
    df['Relative'] = df['Count'] / df['Count'].shift(1)
    df['Relative'].iloc[0] = 1  # First step has 100% relative efficiency
    df['Cumulative'] = df['Count'] / df['Count'].iloc[0]
    return df

# Apply the function to both DataFrames
df_mu = calculate_efficiencies(df_mu)
df_ele = calculate_efficiencies(df_ele)


In [23]:
df_mu

,Count,Relative,Cumulative
Trigger,2710683.0,1.000000,1.000000
bMatch,573591.0,0.211604,0.211604
twoFatJet,90160.0,0.157185,0.033261
oneLep,72510.0,0.804237,0.026750
LepMatch,49585.0,0.683837,0.018292
HadMatch,5717.0,0.115297,0.002109
OneLep,3891.0,0.680602,0.001435


In [24]:
df_ele

,Count,Relative,Cumulative
Trigger,1982680.0,1.000000,1.000000
bMatch,415892.0,0.209763,0.209763
twoFatJet,72616.0,0.174603,0.036625
oneLep,50005.0,0.688622,0.025221
LepMatch,35176.0,0.703450,0.017742
HadMatch,4206.0,0.119570,0.002121
OneLep,3631.0,0.863291,0.001831
